# 🚀 Proyecto: Análisis de Ventas Multicanal
## Databricks Free Edition (Serverless)

### Objetivos del Proyecto:
- Entender la arquitectura de Spark (Driver, Executors, Lazy Evaluation)
- Dominar lectura/escritura en múltiples formatos (CSV, JSON, Parquet)
- Aplicar JOINs complejos, Subqueries y CTEs
- Implementar Window Functions avanzadas
- Crear visualizaciones nativas en Databricks

### ⚠️ Características de Databricks Free Edition:
- **Serverless Compute**: No necesitas crear clusters, el compute se asigna automáticamente
- **Unity Catalog**: Almacenamiento con Volumes (`/Volumes/catalog/schema/volume/`)
- **Lenguajes**: Solo Python y SQL (no R ni Scala)
- **Cuotas**: Límites diarios de uso
- **SQL Warehouse**: 2X-Small incluido

# 🔧 Configuración Inicial

## Crear el Volume para almacenar archivos

Antes de empezar, necesitas crear un Volume en Unity Catalog:

**Opción 1 - Desde la UI:**
1. Ve a **Catalog** en el menú lateral
2. Selecciona tu catálogo (por defecto: `main`)
3. Selecciona tu schema (por defecto: `default`)
4. Clic en **Create** → **Volume**
5. Nombre: `proyecto_ventas`

In [0]:
# Ejecutar en Databricks para crear el Volume
# spark.sql("CREATE VOLUME IF NOT EXISTS main.default.proyecto_ventas")
# spark.sql("SHOW VOLUMES IN main.default").show()

In [0]:
# Definir la ruta base del Volume
# IMPORTANTE: Ajusta esto si tu catálogo/schema son diferentes
CATALOG = "main"
SCHEMA = "default"
VOLUME = "proyecto_ventas"

BASE_PATH = f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}"
print(f"📁 Ruta base del proyecto: {BASE_PATH}")

# 📚 Parte 1: Arquitectura de Apache Spark

## 1.1 Driver y Executors

```
┌─────────────────────────────────────────────────────────────┐
│                      SPARK APPLICATION                       │
├─────────────────────────────────────────────────────────────┤
│  ┌─────────────────┐                                        │
│  │     DRIVER      │  ← Coordina la ejecución               │
│  │  ┌───────────┐  │  ← Mantiene SparkContext               │
│  │  │ SparkCtx  │  │  ← Divide trabajo en Tasks             │
│  │  └───────────┘  │  ← Recolecta resultados                │
│  └────────┬────────┘                                        │
│           │                                                  │
│           ▼                                                  │
│  ┌─────────────────────────────────────────────┐            │
│  │         SERVERLESS RESOURCE MANAGER          │            │
│  │     (Databricks gestiona automáticamente)    │            │
│  └─────────────────────────────────────────────┘            │
│           │                                                  │
│     ┌─────┴─────┬─────────────┐                             │
│     ▼           ▼             ▼                             │
│  ┌──────┐   ┌──────┐     ┌──────┐                           │
│  │ Exec │   │ Exec │     │ Exec │  ← Ejecutan Tasks         │
│  │  1   │   │  2   │     │  N   │  ← Almacenan datos        │
│  └──────┘   └──────┘     └──────┘  ← Procesan particiones   │
└─────────────────────────────────────────────────────────────┘
```

**En Free Edition Serverless**: Databricks asigna recursos automáticamente.

In [0]:
# Verificar la configuración de Spark (Serverless)
print("=" * 60)
print("CONFIGURACIÓN DE SPARK - FREE EDITION SERVERLESS")
print("=" * 60)

# Información del SparkContext
print(f"\n📌 Spark Version: {spark.version}")
print(f"📌 App Name: {spark.sparkContext.appName}")

# Configuraciones disponibles
configs_a_revisar = [
    "spark.sql.shuffle.partitions",
    "spark.databricks.clusterUsageTags.clusterName"
]

print("\n📊 Configuraciones disponibles:")
for config in configs_a_revisar:
    try:
        value = spark.conf.get(config)
        print(f"   {config}: {value}")
    except Exception as e:
        print(f"   {config}: (gestionado por serverless)")

print("\n💡 En Serverless, Databricks optimiza los recursos automáticamente")

## 1.2 Lazy Evaluation - Demostración Práctica

Spark NO ejecuta transformaciones inmediatamente. Solo las ejecuta cuando hay una **acción**.

| Transformaciones (Lazy) | Acciones (Eager) |
|------------------------|------------------|
| select, filter, join   | count, collect   |
| groupBy, orderBy       | show, write      |
| withColumn, drop       | first, take      |

In [0]:
from pyspark.sql.functions import col, lit, when
import time

# Crear un DataFrame para demostrar lazy evaluation
print("🔄 Creando DataFrame...")
df_demo = spark.range(1000000)  # 1 millón de filas

# TRANSFORMACIONES - No se ejecutan aún (LAZY)
print("\n📝 Aplicando transformaciones (LAZY - no se ejecutan aún):")
start = time.time()

df_transformed = (df_demo
    .withColumn("doble", col("id") * 2)
    .withColumn("categoria", when(col("id") < 500000, "A").otherwise("B"))
    .filter(col("id") % 2 == 0)  # Solo pares
    .select("id", "doble", "categoria")
)

print(f"   ⏱️ Tiempo de definición: {time.time() - start:.4f} segundos")
print("   ✅ Las transformaciones están DEFINIDAS pero NO EJECUTADAS")

# ACCIÓN - Aquí SÍ se ejecuta todo el pipeline
print("\n🚀 Ejecutando ACCIÓN (count):")
start = time.time()
total = df_transformed.count()
print(f"   ⏱️ Tiempo de ejecución: {time.time() - start:.4f} segundos")
print(f"   📊 Total de registros: {total:,}")

In [0]:
# Ver el plan de ejecución (Explain Plan)
print("📋 PLAN DE EJECUCIÓN (Explain):")
print("=" * 60)
df_transformed.explain(mode="simple")

# 📁 Parte 2: Generación y Almacenamiento de Datos

In [0]:
from pyspark.sql.functions import (
    to_date, concat, lit, col, 
    count, sum as spark_sum, avg, round as spark_round,
    collect_set, datediff, when
)
from pyspark.sql.types import *
import random

# Configurar semilla para reproducibilidad
random.seed(42)

# ============================================================
# DATOS DE CLIENTES
# ============================================================
print("👥 Generando datos de CLIENTES...")

nombres = ["María", "Juan", "Ana", "Carlos", "Laura", "Pedro", "Sofia", "Diego", "Valentina", "Andrés",
           "Camila", "Luis", "Isabella", "Miguel", "Daniela", "José", "Luciana", "Fernando", "Paula", "Ricardo"]
apellidos = ["García", "Rodríguez", "Martínez", "López", "González", "Hernández", "Pérez", "Sánchez", 
             "Ramírez", "Torres", "Flores", "Rivera", "Gómez", "Díaz", "Reyes", "Cruz", "Morales", "Ortiz"]
ciudades = ["Lima", "Arequipa", "Trujillo", "Cusco", "Piura", "Chiclayo", "Iquitos", "Huancayo"]
segmentos = ["Premium", "Regular", "Básico"]

clientes_data = []
for i in range(1, 501):
    clientes_data.append((
        i,
        random.choice(nombres),
        random.choice(apellidos),
        f"cliente{i}@email.com",
        random.choice(ciudades),
        random.choice(segmentos),
        f"2020-{random.randint(1,12):02d}-{random.randint(1,28):02d}"
    ))

schema_clientes = StructType([
    StructField("cliente_id", IntegerType(), False),
    StructField("nombre", StringType(), True),
    StructField("apellido", StringType(), True),
    StructField("email", StringType(), True),
    StructField("ciudad", StringType(), True),
    StructField("segmento", StringType(), True),
    StructField("fecha_registro", StringType(), True)
])

df_clientes = spark.createDataFrame(clientes_data, schema_clientes)
df_clientes = df_clientes.withColumn("fecha_registro", to_date("fecha_registro"))

print(f"   ✅ {df_clientes.count()} clientes generados")
display(df_clientes.limit(5))

In [0]:
# ============================================================
# DATOS DE PRODUCTOS
# ============================================================
print("📦 Generando datos de PRODUCTOS...")

productos_data = [
    (1, "Laptop HP Pavilion", "Electrónica", "Computadoras", 2499.99, 50),
    (2, "iPhone 15 Pro", "Electrónica", "Smartphones", 4999.99, 30),
    (3, "Samsung Galaxy S24", "Electrónica", "Smartphones", 3799.99, 45),
    (4, "Audífonos Sony WH-1000XM5", "Electrónica", "Audio", 1299.99, 100),
    (5, "Monitor LG 27'' 4K", "Electrónica", "Monitores", 1599.99, 40),
    (6, "Teclado Logitech MX Keys", "Electrónica", "Accesorios", 449.99, 80),
    (7, "Mouse Logitech MX Master", "Electrónica", "Accesorios", 399.99, 90),
    (8, "Tablet iPad Air", "Electrónica", "Tablets", 2799.99, 35),
    (9, "Smartwatch Apple Watch", "Electrónica", "Wearables", 1999.99, 60),
    (10, "Cámara Canon EOS R6", "Electrónica", "Fotografía", 6499.99, 15),
    (11, "Silla Ergonómica", "Muebles", "Oficina", 899.99, 70),
    (12, "Escritorio Standing Desk", "Muebles", "Oficina", 1299.99, 25),
    (13, "Lámpara LED Escritorio", "Muebles", "Iluminación", 149.99, 150),
    (14, "Mochila para Laptop", "Accesorios", "Bolsos", 199.99, 200),
    (15, "Hub USB-C 7 en 1", "Electrónica", "Accesorios", 179.99, 120),
    (16, "Disco SSD 1TB", "Electrónica", "Almacenamiento", 349.99, 85),
    (17, "Webcam Logitech 4K", "Electrónica", "Video", 549.99, 55),
    (18, "Micrófono Blue Yeti", "Electrónica", "Audio", 499.99, 65),
    (19, "Router WiFi 6", "Electrónica", "Redes", 299.99, 75),
    (20, "Power Bank 20000mAh", "Electrónica", "Accesorios", 129.99, 180)
]

schema_productos = StructType([
    StructField("producto_id", IntegerType(), False),
    StructField("nombre_producto", StringType(), True),
    StructField("categoria", StringType(), True),
    StructField("subcategoria", StringType(), True),
    StructField("precio_unitario", DoubleType(), True),
    StructField("stock_actual", IntegerType(), True)
])

df_productos = spark.createDataFrame(productos_data, schema_productos)
print(f"   ✅ {df_productos.count()} productos generados")
display(df_productos.limit(5))

In [0]:
# ============================================================
# DATOS DE TRANSACCIONES (VENTAS)
# ============================================================
print("💰 Generando datos de TRANSACCIONES...")

canales = ["Online", "Tienda Física", "App Móvil", "Marketplace", "Teléfono"]
metodos_pago = ["Tarjeta Crédito", "Tarjeta Débito", "Efectivo", "Transferencia", "PayPal"]
estados = ["Completada", "Completada", "Completada", "Completada", "Pendiente", "Cancelada"]

transacciones_data = []
for i in range(1, 5001):
    cliente_id = random.randint(1, 500)
    producto_id = random.randint(1, 20)
    cantidad = random.randint(1, 5)
    precio = [p[4] for p in productos_data if p[0] == producto_id][0]
    descuento = random.choice([0, 0, 0, 5, 10, 15, 20])
    
    transacciones_data.append((
        i,
        cliente_id,
        producto_id,
        f"2024-{random.randint(1,12):02d}-{random.randint(1,28):02d}",
        cantidad,
        precio,
        descuento,
        random.choice(canales),
        random.choice(metodos_pago),
        random.choice(estados)
    ))

schema_transacciones = StructType([
    StructField("transaccion_id", IntegerType(), False),
    StructField("cliente_id", IntegerType(), True),
    StructField("producto_id", IntegerType(), True),
    StructField("fecha_transaccion", StringType(), True),
    StructField("cantidad", IntegerType(), True),
    StructField("precio_unitario", DoubleType(), True),
    StructField("descuento_porcentaje", IntegerType(), True),
    StructField("canal_venta", StringType(), True),
    StructField("metodo_pago", StringType(), True),
    StructField("estado", StringType(), True)
])

df_transacciones = spark.createDataFrame(transacciones_data, schema_transacciones)
df_transacciones = (df_transacciones
    .withColumn("fecha_transaccion", to_date("fecha_transaccion"))
    .withColumn("total_bruto", col("cantidad") * col("precio_unitario"))
    .withColumn("monto_descuento", col("total_bruto") * col("descuento_porcentaje") / 100)
    .withColumn("total_neto", col("total_bruto") - col("monto_descuento"))
)

print(f"   ✅ {df_transacciones.count()} transacciones generadas")
display(df_transacciones.limit(5))

## 2.1 Escritura de Datos en Múltiples Formatos

En Free Edition, usamos **Unity Catalog Volumes** para almacenar archivos.

In [0]:
from pyspark.sql.functions import month

print(f"📁 Guardando datos en: {BASE_PATH}")
print("=" * 60)

# ============================================================
# GUARDAR EN CSV
# ============================================================
csv_path = f"{BASE_PATH}/csv"

df_clientes.write.mode("overwrite").option("header", True).csv(f"{csv_path}/clientes")
df_productos.write.mode("overwrite").option("header", True).csv(f"{csv_path}/productos")
df_transacciones.write.mode("overwrite").option("header", True).csv(f"{csv_path}/transacciones")

print(f"✅ CSV guardado en: {csv_path}")

# ============================================================
# GUARDAR EN JSON
# ============================================================
json_path = f"{BASE_PATH}/json"

df_clientes.write.mode("overwrite").json(f"{json_path}/clientes")
df_productos.write.mode("overwrite").json(f"{json_path}/productos")
df_transacciones.write.mode("overwrite").json(f"{json_path}/transacciones")

print(f"✅ JSON guardado en: {json_path}")

# ============================================================
# GUARDAR EN PARQUET (Formato más eficiente)
# ============================================================
parquet_path = f"{BASE_PATH}/parquet"

df_clientes.write.mode("overwrite").parquet(f"{parquet_path}/clientes")
df_productos.write.mode("overwrite").parquet(f"{parquet_path}/productos")

# Transacciones particionadas por mes
df_transacciones_part = df_transacciones.withColumn("mes", month("fecha_transaccion"))
df_transacciones_part.write.mode("overwrite").partitionBy("mes").parquet(f"{parquet_path}/transacciones")

print(f"✅ Parquet guardado en: {parquet_path}")
print(f"   📌 Transacciones particionadas por MES")

## 2.2 Lectura de Datos desde Diferentes Formatos

In [0]:
# ============================================================
# LECTURA DESDE CSV
# ============================================================
print("📖 Leyendo desde CSV...")

clientes_csv = (spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv(f"{csv_path}/clientes")
)

print(f"   Registros leídos: {clientes_csv.count()}")
print(f"   Schema inferido:")
clientes_csv.printSchema()

In [0]:
# ============================================================
# LECTURA DESDE JSON
# ============================================================
print("📖 Leyendo desde JSON...")

productos_json = spark.read.json(f"{json_path}/productos")

print(f"   Registros leídos: {productos_json.count()}")
display(productos_json.limit(3))

In [0]:
# ============================================================
# LECTURA DESDE PARQUET (Más eficiente)
# ============================================================
print("📖 Leyendo desde PARQUET...")

transacciones_parquet = spark.read.parquet(f"{parquet_path}/transacciones")

print(f"   Registros leídos: {transacciones_parquet.count()}")

# Ver las particiones
print(f"\n📊 Estructura de particiones por mes:")
display(transacciones_parquet.select("mes").distinct().orderBy("mes"))

In [0]:
# Comparación de rendimiento entre formatos
import time

def medir_lectura(formato, path):
    start = time.time()
    if formato == "csv":
        df = spark.read.option("header", True).option("inferSchema", True).csv(path)
    elif formato == "json":
        df = spark.read.json(path)
    else:
        df = spark.read.parquet(path)
    
    count = df.count()
    elapsed = time.time() - start
    return elapsed, count

print("⏱️ COMPARACIÓN DE TIEMPOS DE LECTURA")
print("=" * 60)

formatos = [
    ("CSV", "csv", f"{csv_path}/transacciones"),
    ("JSON", "json", f"{json_path}/transacciones"),
    ("Parquet", "parquet", f"{parquet_path}/transacciones")
]

for nombre, formato, path in formatos:
    tiempo, registros = medir_lectura(formato, path)
    print(f"   {nombre:10} → {tiempo:.3f} seg ({registros:,} registros)")

print("\n💡 Parquet es generalmente más rápido por su formato columnar y compresión")

# 🔗 Parte 3: JOINs Complejos, Subqueries y CTEs

In [0]:
# Cargar datos desde Parquet para el análisis
df_clientes = spark.read.parquet(f"{parquet_path}/clientes")
df_productos = spark.read.parquet(f"{parquet_path}/productos")
df_transacciones = spark.read.parquet(f"{parquet_path}/transacciones")

# Crear vistas temporales para SQL
df_clientes.createOrReplaceTempView("clientes")
df_productos.createOrReplaceTempView("productos")
df_transacciones.createOrReplaceTempView("transacciones")

print("✅ Vistas SQL creadas: clientes, productos, transacciones")

## 3.1 JOINs Básicos y Complejos

In [0]:
# ============================================================
# INNER JOIN - Transacciones con información completa
# ============================================================
print("🔗 INNER JOIN - Vista completa de ventas")

df_ventas_completas = (df_transacciones
    .join(df_clientes, "cliente_id", "inner")
    .join(df_productos, "producto_id", "inner")
    .select(
        "transaccion_id",
        "fecha_transaccion",
        concat(col("nombre"), lit(" "), col("apellido")).alias("cliente"),
        "ciudad",
        "segmento",
        "nombre_producto",
        "categoria",
        "canal_venta",
        "cantidad",
        "total_neto",
        "estado"
    )
)

display(df_ventas_completas.limit(10))

In [0]:
# ============================================================
# LEFT JOIN - Clientes con o sin compras
# ============================================================
print("🔗 LEFT JOIN - Análisis de clientes (con y sin compras)")

# Resumen de compras por cliente
df_resumen_cliente = (df_transacciones
    .filter(col("estado") == "Completada")
    .groupBy("cliente_id")
    .agg(
        count("transaccion_id").alias("total_compras"),
        spark_round(spark_sum("total_neto"), 2).alias("total_gastado"),
        spark_round(avg("total_neto"), 2).alias("ticket_promedio")
    )
)

# LEFT JOIN para incluir clientes sin compras
df_clientes_analisis = (df_clientes
    .join(df_resumen_cliente, "cliente_id", "left")
    .fillna(0, ["total_compras", "total_gastado", "ticket_promedio"])
    .orderBy(col("total_gastado").desc())
)

print(f"Total clientes: {df_clientes_analisis.count()}")
print(f"Clientes sin compras: {df_clientes_analisis.filter(col('total_compras') == 0).count()}")
display(df_clientes_analisis.limit(10))

In [0]:
# ============================================================
# Análisis de clientes multicanal
# ============================================================
print("🔗 Análisis de clientes multicanal")

from pyspark.sql.functions import countDistinct

df_multicanal = (df_transacciones
    .filter(col("estado") == "Completada")
    .groupBy("cliente_id")
    .agg(
        countDistinct("canal_venta").alias("canales_usados"),
        collect_set("canal_venta").alias("lista_canales")
    )
    .filter(col("canales_usados") > 1)
    .join(df_clientes.select("cliente_id", "nombre", "apellido", "segmento"), "cliente_id")
    .orderBy(col("canales_usados").desc())
)

print(f"Clientes que compran en múltiples canales: {df_multicanal.count()}")
display(df_multicanal.limit(10))

## 3.2 Subqueries y CTEs en SQL

In [0]:
# ============================================================
# SUBQUERY en WHERE - Clientes con compras mayores al promedio
# ============================================================

df_subquery = spark.sql("""
    SELECT 
        c.cliente_id,
        c.nombre,
        c.apellido,
        c.ciudad,
        c.segmento,
        ROUND(SUM(t.total_neto), 2) as total_gastado
    FROM clientes c
    INNER JOIN transacciones t ON c.cliente_id = t.cliente_id
    WHERE t.estado = 'Completada'
    GROUP BY c.cliente_id, c.nombre, c.apellido, c.ciudad, c.segmento
    HAVING SUM(t.total_neto) > (
        SELECT AVG(total_cliente)
        FROM (
            SELECT cliente_id, SUM(total_neto) as total_cliente
            FROM transacciones
            WHERE estado = 'Completada'
            GROUP BY cliente_id
        )
    )
    ORDER BY total_gastado DESC
    LIMIT 15
""")

print("📊 SUBQUERY - Clientes con compras mayores al promedio:")
display(df_subquery)

In [0]:
# ============================================================
# CTE (Common Table Expression) - Análisis jerárquico de ventas
# ============================================================

df_cte = spark.sql("""
    WITH ventas_por_categoria AS (
        SELECT 
            p.categoria,
            COUNT(t.transaccion_id) as num_ventas,
            ROUND(SUM(t.total_neto), 2) as ingresos_totales,
            ROUND(AVG(t.total_neto), 2) as ticket_promedio
        FROM transacciones t
        INNER JOIN productos p ON t.producto_id = p.producto_id
        WHERE t.estado = 'Completada'
        GROUP BY p.categoria
    ),

    ventas_por_canal AS (
        SELECT 
            p.categoria,
            t.canal_venta,
            ROUND(SUM(t.total_neto), 2) as ingresos_canal
        FROM transacciones t
        INNER JOIN productos p ON t.producto_id = p.producto_id
        WHERE t.estado = 'Completada'
        GROUP BY p.categoria, t.canal_venta
    ),

    ranking_canales AS (
        SELECT 
            categoria,
            canal_venta,
            ingresos_canal,
            ROW_NUMBER() OVER (PARTITION BY categoria ORDER BY ingresos_canal DESC) as ranking
        FROM ventas_por_canal
    )

    SELECT 
        vp.categoria,
        vp.num_ventas,
        vp.ingresos_totales,
        vp.ticket_promedio,
        rc.canal_venta as canal_principal,
        rc.ingresos_canal as ingresos_canal_principal
    FROM ventas_por_categoria vp
    INNER JOIN ranking_canales rc ON vp.categoria = rc.categoria AND rc.ranking = 1
    ORDER BY vp.ingresos_totales DESC
""")

print("📊 CTE - Análisis jerárquico de ventas:")
display(df_cte)

# 🪟 Parte 4: Window Functions

## 4.1 ROW_NUMBER, RANK, DENSE_RANK

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import (
    row_number, rank, dense_rank, 
    lag, lead, 
    sum as spark_sum, avg as spark_avg,
    first, last, ntile, date_format
)

In [0]:
# ============================================================
# ROW_NUMBER - Numerar transacciones por cliente
# ============================================================
print("🔢 ROW_NUMBER - Orden de compras por cliente")

window_cliente = Window.partitionBy("cliente_id").orderBy(col("fecha_transaccion").desc())

df_con_numero = (df_transacciones
    .filter(col("estado") == "Completada")
    .withColumn("numero_compra", row_number().over(window_cliente))
    .join(df_clientes.select("cliente_id", "nombre", "apellido"), "cliente_id")
    .select("cliente_id", "nombre", "apellido", "fecha_transaccion", "total_neto", "numero_compra")
    .filter(col("numero_compra") <= 3)
    .orderBy("cliente_id", "numero_compra")
)

display(df_con_numero.limit(15))

In [0]:
# ============================================================
# RANK vs DENSE_RANK - Ranking de productos por ingresos
# ============================================================
print("🏆 RANK vs DENSE_RANK - Comparación de rankings")

df_ingresos_producto = (df_transacciones
    .filter(col("estado") == "Completada")
    .join(df_productos.select("producto_id", "nombre_producto", "categoria"), "producto_id")
    .groupBy("producto_id", "nombre_producto", "categoria")
    .agg(spark_round(spark_sum("total_neto"), 2).alias("ingresos_totales"))
)

window_categoria = Window.partitionBy("categoria").orderBy(col("ingresos_totales").desc())

df_ranking = (df_ingresos_producto
    .withColumn("rank", rank().over(window_categoria))
    .withColumn("dense_rank", dense_rank().over(window_categoria))
    .withColumn("row_number", row_number().over(window_categoria))
    .orderBy("categoria", "rank")
)

print("📌 Diferencias:")
print("   - ROW_NUMBER: Siempre único (1,2,3,4,5...)")
print("   - RANK: Permite empates, salta posiciones (1,2,2,4,5...)")
print("   - DENSE_RANK: Permite empates, no salta (1,2,2,3,4...)")

display(df_ranking.limit(20))

## 4.2 LAG y LEAD - Análisis Temporal

In [0]:
# ============================================================
# LAG y LEAD - Comparar ventas entre periodos
# ============================================================
print("📈 LAG/LEAD - Análisis de tendencias de ventas mensuales")

# Ventas mensuales
df_ventas_mes = (df_transacciones
    .filter(col("estado") == "Completada")
    .withColumn("anio_mes", date_format("fecha_transaccion", "yyyy-MM"))
    .groupBy("anio_mes")
    .agg(
        count("transaccion_id").alias("num_ventas"),
        spark_round(spark_sum("total_neto"), 2).alias("ingresos")
    )
    .orderBy("anio_mes")
)

window_temporal = Window.orderBy("anio_mes")

df_tendencias = (df_ventas_mes
    .withColumn("ingresos_mes_anterior", lag("ingresos", 1).over(window_temporal))
    .withColumn("ingresos_mes_siguiente", lead("ingresos", 1).over(window_temporal))
    .withColumn("variacion_vs_anterior", 
        spark_round((col("ingresos") - col("ingresos_mes_anterior")) / col("ingresos_mes_anterior") * 100, 2))
    .withColumn("tendencia", 
        when(col("variacion_vs_anterior") > 5, "📈 Crecimiento")
        .when(col("variacion_vs_anterior") < -5, "📉 Decrecimiento")
        .otherwise("➡️ Estable"))
)

display(df_tendencias)

In [0]:
# ============================================================
# LAG - Análisis de frecuencia de compra por cliente
# ============================================================
print("🔄 LAG - Días entre compras por cliente")

window_compras = Window.partitionBy("cliente_id").orderBy("fecha_transaccion")

df_frecuencia = (df_transacciones
    .filter(col("estado") == "Completada")
    .withColumn("fecha_compra_anterior", lag("fecha_transaccion", 1).over(window_compras))
    .withColumn("dias_entre_compras", 
        datediff("fecha_transaccion", "fecha_compra_anterior"))
    .filter(col("dias_entre_compras").isNotNull())
    .join(df_clientes.select("cliente_id", "nombre", "segmento"), "cliente_id")
)

# Promedio de días entre compras por segmento
df_frecuencia_segmento = (df_frecuencia
    .groupBy("segmento")
    .agg(
        spark_round(spark_avg("dias_entre_compras"), 1).alias("promedio_dias_entre_compras"),
        count("*").alias("total_recompras")
    )
    .orderBy("promedio_dias_entre_compras")
)

print("⏱️ Frecuencia de recompra por segmento:")
display(df_frecuencia_segmento)

## 4.3 Running Totals y Moving Averages

In [0]:
# ============================================================
# RUNNING TOTAL - Ventas acumuladas
# ============================================================
print("📊 Running Total - Ingresos acumulados por mes")

window_acumulado = Window.orderBy("anio_mes").rowsBetween(Window.unboundedPreceding, Window.currentRow)

df_acumulado = (df_ventas_mes
    .withColumn("ingresos_acumulados", spark_round(spark_sum("ingresos").over(window_acumulado), 2))
    .withColumn("ventas_acumuladas", spark_sum("num_ventas").over(window_acumulado))
)

display(df_acumulado)

In [0]:
# ============================================================
# MOVING AVERAGE - Promedio móvil de 3 meses
# ============================================================
print("📉 Moving Average - Promedio móvil de ingresos (3 meses)")

window_moving = Window.orderBy("anio_mes").rowsBetween(-2, 0)

df_moving_avg = (df_ventas_mes
    .withColumn("promedio_movil_3m", spark_round(spark_avg("ingresos").over(window_moving), 2))
    .withColumn("diferencia_vs_promedio", 
        spark_round(col("ingresos") - col("promedio_movil_3m"), 2))
)

display(df_moving_avg)

In [0]:
# ============================================================
# Window Functions completas en SQL
# ============================================================

df_window_sql = spark.sql("""
    WITH ventas_diarias AS (
        SELECT 
            fecha_transaccion,
            canal_venta,
            COUNT(*) as num_ventas,
            ROUND(SUM(total_neto), 2) as ingresos
        FROM transacciones
        WHERE estado = 'Completada'
        GROUP BY fecha_transaccion, canal_venta
    )

    SELECT 
        fecha_transaccion,
        canal_venta,
        num_ventas,
        ingresos,
        RANK() OVER (PARTITION BY fecha_transaccion ORDER BY ingresos DESC) as ranking_dia,
        LAG(ingresos, 1) OVER (PARTITION BY canal_venta ORDER BY fecha_transaccion) as ingresos_dia_anterior,
        SUM(ingresos) OVER (PARTITION BY canal_venta ORDER BY fecha_transaccion 
                            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as acumulado_canal,
        ROUND(ingresos * 100.0 / SUM(ingresos) OVER (PARTITION BY fecha_transaccion), 2) as pct_del_dia
    FROM ventas_diarias
    ORDER BY fecha_transaccion DESC, ranking_dia
    LIMIT 30
""")

print("📊 Window Functions en SQL:")
display(df_window_sql)

# 📊 Parte 5: Visualizaciones Nativas en Databricks

## Cómo crear visualizaciones:
1. Ejecuta la celda con `display(df)`
2. Clic en el icono **"+"** junto a la tabla
3. Selecciona **"Visualization"**
4. Elige el tipo de gráfico y configura los ejes

In [0]:
# ============================================================
# VISUALIZACIÓN 1: Ventas por Canal (Gráfico de Barras)
# ============================================================
# Instrucción: Clic en "+" → Visualization → Bar Chart
# X-axis: canal_venta | Y-axis: ingresos_totales

df_ventas_canal = spark.sql("""
    SELECT 
        canal_venta,
        COUNT(*) as numero_ventas,
        ROUND(SUM(total_neto), 2) as ingresos_totales,
        ROUND(AVG(total_neto), 2) as ticket_promedio
    FROM transacciones
    WHERE estado = 'Completada'
    GROUP BY canal_venta
    ORDER BY ingresos_totales DESC
""")

display(df_ventas_canal)

In [0]:
# ============================================================
# VISUALIZACIÓN 2: Tendencia Mensual (Gráfico de Líneas)
# ============================================================
# Instrucción: Line Chart con anio_mes en X-axis

df_tendencia_mensual = spark.sql("""
    SELECT 
        DATE_FORMAT(fecha_transaccion, 'yyyy-MM') as anio_mes,
        COUNT(*) as numero_ventas,
        ROUND(SUM(total_neto), 2) as ingresos,
        COUNT(DISTINCT cliente_id) as clientes_activos
    FROM transacciones
    WHERE estado = 'Completada'
    GROUP BY DATE_FORMAT(fecha_transaccion, 'yyyy-MM')
    ORDER BY anio_mes
""")

display(df_tendencia_mensual)

In [0]:
# ============================================================
# VISUALIZACIÓN 3: Distribución por Categoría (Pie Chart)
# ============================================================
# Instrucción: Pie Chart con categoria como keys

df_categoria = spark.sql("""
    SELECT 
        p.categoria,
        COUNT(*) as numero_ventas,
        ROUND(SUM(t.total_neto), 2) as ingresos,
        ROUND(SUM(t.total_neto) * 100.0 / (SELECT SUM(total_neto) FROM transacciones WHERE estado = 'Completada'), 2) as porcentaje
    FROM transacciones t
    JOIN productos p ON t.producto_id = p.producto_id
    WHERE t.estado = 'Completada'
    GROUP BY p.categoria
    ORDER BY ingresos DESC
""")

display(df_categoria)

In [0]:
# ============================================================
# VISUALIZACIÓN 4: Top 10 Productos
# ============================================================

df_top_productos = spark.sql("""
    SELECT 
        p.nombre_producto,
        p.categoria,
        COUNT(*) as unidades_vendidas,
        ROUND(SUM(t.total_neto), 2) as ingresos
    FROM transacciones t
    JOIN productos p ON t.producto_id = p.producto_id
    WHERE t.estado = 'Completada'
    GROUP BY p.producto_id, p.nombre_producto, p.categoria
    ORDER BY ingresos DESC
    LIMIT 10
""")

display(df_top_productos)

In [0]:
# ============================================================
# VISUALIZACIÓN 5: Análisis por Segmento de Cliente
# ============================================================

df_segmento_canal = spark.sql("""
    SELECT 
        c.segmento,
        t.canal_venta,
        COUNT(*) as numero_ventas,
        ROUND(SUM(t.total_neto), 2) as ingresos,
        ROUND(AVG(t.total_neto), 2) as ticket_promedio
    FROM transacciones t
    JOIN clientes c ON t.cliente_id = c.cliente_id
    WHERE t.estado = 'Completada'
    GROUP BY c.segmento, t.canal_venta
    ORDER BY c.segmento, ingresos DESC
""")

display(df_segmento_canal)

In [0]:
# ============================================================
# VISUALIZACIÓN 6: Ventas por Día de la Semana
# ============================================================

df_dia_semana = spark.sql("""
    SELECT 
        DAYOFWEEK(fecha_transaccion) as dia_num,
        CASE DAYOFWEEK(fecha_transaccion)
            WHEN 1 THEN 'Domingo'
            WHEN 2 THEN 'Lunes'
            WHEN 3 THEN 'Martes'
            WHEN 4 THEN 'Miércoles'
            WHEN 5 THEN 'Jueves'
            WHEN 6 THEN 'Viernes'
            WHEN 7 THEN 'Sábado'
        END as dia_semana,
        COUNT(*) as ventas,
        ROUND(SUM(total_neto), 2) as ingresos
    FROM transacciones
    WHERE estado = 'Completada'
    GROUP BY DAYOFWEEK(fecha_transaccion)
    ORDER BY dia_num
""")

display(df_dia_semana)

In [0]:
# ============================================================
# VISUALIZACIÓN 7: Métodos de Pago
# ============================================================

df_metodo_pago = spark.sql("""
    SELECT 
        metodo_pago,
        COUNT(*) as transacciones,
        ROUND(SUM(total_neto), 2) as ingresos,
        ROUND(AVG(total_neto), 2) as ticket_promedio
    FROM transacciones
    WHERE estado = 'Completada'
    GROUP BY metodo_pago
    ORDER BY transacciones DESC
""")

display(df_metodo_pago)

In [0]:
# ============================================================
# VISUALIZACIÓN 8: Ventas por Ciudad
# ============================================================

df_ciudades = spark.sql("""
    SELECT 
        c.ciudad,
        COUNT(DISTINCT c.cliente_id) as clientes,
        COUNT(t.transaccion_id) as transacciones,
        ROUND(SUM(t.total_neto), 2) as ingresos,
        ROUND(SUM(t.total_neto) / COUNT(DISTINCT c.cliente_id), 2) as valor_por_cliente
    FROM clientes c
    LEFT JOIN transacciones t ON c.cliente_id = t.cliente_id AND t.estado = 'Completada'
    GROUP BY c.ciudad
    ORDER BY ingresos DESC
""")

display(df_ciudades)

# 🎯 Parte 6: Proyecto Final - Análisis Avanzado

In [0]:
# ============================================================
# ANÁLISIS RFM (Recency, Frequency, Monetary)
# ============================================================
print("📊 ANÁLISIS RFM - Segmentación de Clientes")

df_rfm = spark.sql("""
    WITH cliente_metricas AS (
        SELECT 
            t.cliente_id,
            DATEDIFF(CURRENT_DATE(), MAX(t.fecha_transaccion)) as recency,
            COUNT(DISTINCT t.transaccion_id) as frequency,
            ROUND(SUM(t.total_neto), 2) as monetary
        FROM transacciones t
        WHERE t.estado = 'Completada'
        GROUP BY t.cliente_id
    ),
    
    rfm_scores AS (
        SELECT 
            cliente_id,
            recency,
            frequency,
            monetary,
            NTILE(5) OVER (ORDER BY recency DESC) as r_score,
            NTILE(5) OVER (ORDER BY frequency) as f_score,
            NTILE(5) OVER (ORDER BY monetary) as m_score
        FROM cliente_metricas
    )
    
    SELECT 
        r.*,
        CONCAT(r_score, f_score, m_score) as rfm_segment,
        CASE 
            WHEN r_score >= 4 AND f_score >= 4 AND m_score >= 4 THEN 'Champions'
            WHEN r_score >= 4 AND f_score >= 3 THEN 'Loyal Customers'
            WHEN r_score >= 4 AND f_score <= 2 THEN 'New Customers'
            WHEN r_score <= 2 AND f_score >= 3 AND m_score >= 3 THEN 'At Risk'
            WHEN r_score <= 2 AND f_score <= 2 THEN 'Lost'
            ELSE 'Regular'
        END as customer_segment
    FROM rfm_scores r
""")

# Resumen de segmentos
df_rfm_resumen = (df_rfm
    .groupBy("customer_segment")
    .agg(
        count("*").alias("num_clientes"),
        spark_round(spark_avg("monetary"), 2).alias("valor_promedio"),
        spark_round(spark_avg("frequency"), 1).alias("frecuencia_promedio")
    )
    .orderBy(col("num_clientes").desc())
)

display(df_rfm_resumen)

In [0]:
# ============================================================
# ANÁLISIS DE COHORTS
# ============================================================
print("📈 ANÁLISIS DE COHORTS - Retención por mes de primera compra")

df_cohorts = spark.sql("""
    WITH primera_compra AS (
        SELECT 
            cliente_id,
            DATE_FORMAT(MIN(fecha_transaccion), 'yyyy-MM') as cohort_mes
        FROM transacciones
        WHERE estado = 'Completada'
        GROUP BY cliente_id
    ),
    
    actividad_mensual AS (
        SELECT 
            t.cliente_id,
            pc.cohort_mes,
            DATE_FORMAT(t.fecha_transaccion, 'yyyy-MM') as mes_actividad
        FROM transacciones t
        JOIN primera_compra pc ON t.cliente_id = pc.cliente_id
        WHERE t.estado = 'Completada'
    )
    
    SELECT 
        cohort_mes,
        mes_actividad,
        COUNT(DISTINCT cliente_id) as clientes_activos
    FROM actividad_mensual
    GROUP BY cohort_mes, mes_actividad
    ORDER BY cohort_mes, mes_actividad
""")

display(df_cohorts)

In [0]:
# ============================================================
# REPORTE EJECUTIVO FINAL
# ============================================================

df_reporte = spark.sql("""
    SELECT '📊 RESUMEN EJECUTIVO - VENTAS MULTICANAL' as reporte
    UNION ALL SELECT '═══════════════════════════════════════════════════'
    UNION ALL 
    SELECT CONCAT('Total Ingresos: S/ ', FORMAT_NUMBER(SUM(total_neto), 2))
    FROM transacciones WHERE estado = 'Completada'
    UNION ALL
    SELECT CONCAT('Total Transacciones: ', FORMAT_NUMBER(COUNT(*), 0))
    FROM transacciones WHERE estado = 'Completada'
    UNION ALL
    SELECT CONCAT('Clientes Únicos: ', FORMAT_NUMBER(COUNT(DISTINCT cliente_id), 0))
    FROM transacciones WHERE estado = 'Completada'
    UNION ALL
    SELECT CONCAT('Ticket Promedio: S/ ', FORMAT_NUMBER(AVG(total_neto), 2))
    FROM transacciones WHERE estado = 'Completada'
""")

display(df_reporte)

# ✅ Checklist de Temas Cubiertos

| Tema | Estado | Sección |
|------|--------|--------|
| Arquitectura Spark (Driver, Executors) | ✅ | Parte 1.1 |
| Lazy Evaluation | ✅ | Parte 1.2 |
| Lectura/Escritura CSV | ✅ | Parte 2 |
| Lectura/Escritura JSON | ✅ | Parte 2 |
| Lectura/Escritura Parquet | ✅ | Parte 2 |
| INNER JOIN | ✅ | Parte 3.1 |
| LEFT JOIN | ✅ | Parte 3.1 |
| Análisis Multicanal | ✅ | Parte 3.1 |
| Subqueries | ✅ | Parte 3.2 |
| CTEs (WITH clause) | ✅ | Parte 3.2 |
| ROW_NUMBER | ✅ | Parte 4.1 |
| RANK / DENSE_RANK | ✅ | Parte 4.1 |
| LAG | ✅ | Parte 4.2 |
| LEAD | ✅ | Parte 4.2 |
| Running Totals | ✅ | Parte 4.3 |
| Moving Averages | ✅ | Parte 4.3 |
| Visualizaciones Nativas | ✅ | Parte 5 |
| Proyecto Integrado (RFM, Cohorts) | ✅ | Parte 6 |

In [0]:
print("""
╔══════════════════════════════════════════════════════════════════╗
║                    🎉 PROYECTO COMPLETADO 🎉                     ║
╠══════════════════════════════════════════════════════════════════╣
║                                                                  ║
║  Has completado el proyecto de Análisis de Ventas Multicanal    ║
║  en Databricks Free Edition (Serverless).                       ║
║                                                                  ║
║  Características usadas:                                        ║
║  ✓ Serverless Compute (automático)                              ║
║  ✓ Unity Catalog Volumes para archivos                          ║
║  ✓ Python y SQL                                                 ║
║  ✓ Visualizaciones nativas                                      ║
║                                                                  ║
╚══════════════════════════════════════════════════════════════════╝
""")